In [1]:
!pip install evaluate
!pip install datasets
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece
!pip install rouge_score
!pip install accelerate -U

ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8390a06ee0276fa3b9482788fc16f0225ba413177aea00f115459f55fedb8f3c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.3 MB/s eta 0:00:00


In [2]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

In [3]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [5]:
squad_dataset = load_dataset("squad")
train_dataset = squad_dataset["train"]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [18]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [17]:
from datasets import Dataset as Ds

In [10]:
temp=pd.DataFrame(train_dataset)

In [44]:
newdf=temp.iloc[:10000,:]
evaldf=temp.iloc[10000:10500,:]
data=Ds.from_pandas(newdf)
evaldata=Ds.from_pandas(evaldf)

In [45]:
class SquadDataset(Dataset):
    def __init__(self, squad_examples, tokenizer):
        self.squad_examples = squad_examples
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.squad_examples)

    def __getitem__(self, idx):
        example = self.squad_examples[idx]
        inputs = self.tokenizer(
            example["context"],
            example["question"],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512,
        )
        answer_start = example["answers"]["answer_start"][0]
        answer_end = answer_start + len(example["answers"]["text"][0])
        labels = tokenizer(
            example["answers"]["text"][0],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=32,  # Adjust as needed based on your maximum answer length
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
        }

custom_dataset = SquadDataset(data, tokenizer)
eval_dataset=SquadDataset(evaldata,tokenizer)

In [46]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=custom_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.064200,0.064678,0.819059,0.532125,0.818607,0.817924
2,0.028600,0.066335,0.826019,0.527511,0.825833,0.825255


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TrainOutput(global_step=2500, training_loss=0.04707575340270996, metrics={'train_runtime': 3093.4378, 'train_samples_per_second': 6.465, 'train_steps_per_second': 0.808, 'total_flos': 1.369514704896e+16, 'train_loss': 0.04707575340270996, 'epoch': 2.0})

In [47]:
fine_tuned=trainer.model
import torch

In [66]:
def predict_answer(context,question):
  inputs=tokenizer(
            context,
            question,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512,
        )
  with torch.no_grad():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    output = fine_tuned.generate(**inputs)
    return output

context = """
India had won world cup in 2011 and lost in 2015 and 2019 despite playing good in the league stage and the phenomenal batting performances from kohli and rohit
"""

question = "did inida played well in league stages"


ans=predict_answer(context,question)
ans= tokenizer.decode(ans[0], skip_special_tokens=True)
print(ans)

inida played good in league stages


In [51]:
torch.save(fine_tuned.state_dict(), "./fine_tuned_T5.pth")